In [4]:
import duckdb

users_dir = "/home/muthu/Desktop/New Folder/data/users/part-00001.parquet"
events_dir = "/home/muthu/Desktop/New Folder/data/events/part-00001.parquet"

con = duckdb.connect()

# Register parquet datasets
con.execute(f"CREATE VIEW users AS SELECT id, location FROM '{users_dir}'")
con.execute(f"""
    CREATE VIEW events AS
    SELECT
        muid,         -- adjust if column is mid/muid
        session_id,
        event_name,
        device_os,
        channel,
        traffic_source,
        CASE WHEN product_id = '' THEN 'None' ELSE product_id END AS product_id,
        category
    FROM '{events_dir}'
""")

# Join + aggregate into features
con.execute("""
    COPY (
        SELECT
            e.muid,

            COUNT(*) AS event_count,

            SUM(CASE WHEN e.event_name = 'purchase'    THEN 1 ELSE 0 END) AS purchase_count,
            SUM(CASE WHEN e.event_name = 'add_to_cart' THEN 1 ELSE 0 END) AS add_to_cart_count,
            SUM(CASE WHEN e.event_name = 'viewcontent' THEN 1 ELSE 0 END) AS viewcontent_count,
            SUM(CASE WHEN e.event_name = 'scroll'      THEN 1 ELSE 0 END) AS scroll_count,
            SUM(CASE WHEN e.event_name = 'search'      THEN 1 ELSE 0 END) AS search_count,
            SUM(CASE WHEN e.event_name = 'pageview'    THEN 1 ELSE 0 END) AS pageview_count,

            SUM(CASE WHEN e.device_os = 'desktop' THEN 1 ELSE 0 END) AS desktop_count,
            SUM(CASE WHEN e.device_os = 'android' THEN 1 ELSE 0 END) AS android_count,
            SUM(CASE WHEN e.device_os = 'ios'     THEN 1 ELSE 0 END) AS ios_count,

            SUM(CASE WHEN e.channel = 'browser' THEN 1 ELSE 0 END) AS browser_count,
            SUM(CASE WHEN e.channel = 'app'     THEN 1 ELSE 0 END) AS app_count,

            SUM(CASE WHEN e.traffic_source = 'seo'    THEN 1 ELSE 0 END) AS seo_count,
            SUM(CASE WHEN e.traffic_source = 'social' THEN 1 ELSE 0 END) AS social_count,
            SUM(CASE WHEN e.traffic_source = 'paid'   THEN 1 ELSE 0 END) AS paid_count,
            SUM(CASE WHEN e.traffic_source = 'direct' THEN 1 ELSE 0 END) AS direct_count,

            COUNT(DISTINCT e.session_id) AS session_count,

            COUNT(DISTINCT e.product_id) AS unique_products,
            COUNT(DISTINCT e.category)   AS unique_categories,

            COUNT(DISTINCT u.location)   AS unique_locations

        FROM events e
        JOIN users u ON e.muid = u.id
        GROUP BY e.muid
    )
    TO 'features.parquet' (FORMAT PARQUET);
""")
